In [1]:
#Importing modules
import numpy as np
from create_models import CreateModels
import math

# Input data

The input data for the damage model is introduced in the cell below.
In this case, the model corresponds to a crack propagation law, where a crack grows over time.

In [2]:
config = {}

config["T"] = 30 # Finite horizon in years
config["ncycles"] = 1000000 # Number of load cycle experienced in one year
config["d0_mean"] = 1 # Initial crack size
config["dcrit"] = 20  # Critical crack size
config["S_mean"] = 70 # Stress range definition in MPa
config["S_std"] = 10 # Crack growth variable
config["m"] = 3.5 # Crack growth variable
config["lnC_mean"] = -35.2 # Crack growth variable
config["lnC_std"] = 0.5 # Crack growth variable
config

{'T': 30,
 'ncycles': 1000000,
 'd0_mean': 1,
 'dcrit': 20,
 'S_mean': 70,
 'S_std': 10,
 'm': 3.5,
 'lnC_mean': -35.2,
 'lnC_std': 0.5}

# Simulations

An object is loaded taking into account the previously defined damage model.
This is done by calling the class "Pomdp": Pomdp(config)

In [3]:
model_dr = CreateModels(config)

The crack size distribution is propagated over time in a Monte Carlo simuation here below.
This is done through the method crack_growth(n_samples), where the number of samples are specified.
The output consists of crack growth results for the horizon and samples.

In [4]:
n_samples = 1e6
model_dr.crack_growth(n_samples)

C:\coding\tbd\imp_marl\imp_env\pomdp_models\create_models.py:30: RuntimeWarning: invalid value encountered in power
  dt = (((2 - self.m) / 2) * C * (


In [5]:
model_dr.dd.shape

(31, 1000000)

# Transition model

The transition model is below constructed by calling the method transition_model().
Note that this follows a particular discretisation scheme.

The output provides: 
* Inital damage probabilities: model_dr.b0
* Transition probabilities for a do-nothing action: model_dr.T0
* Transition probabilities for a repair action: model_dr.Tr

In [6]:
model_dr.transition_model()

array([1.00000000e-100, 1.00000000e-004, 1.54640235e-004, 2.39136022e-004,
       3.69800505e-004, 5.71860368e-004, 8.84326215e-004, 1.36752413e-003,
       2.11474253e-003, 3.27024280e-003, 5.05711114e-003, 7.82032854e-003,
       1.20933744e-002, 1.87012225e-002, 2.89196144e-002, 4.47213595e-002,
       6.91572153e-002, 1.06944880e-001, 1.65379813e-001, 2.55743731e-001,
       3.95482706e-001, 6.11575384e-001, 9.45741609e-001, 1.46249704e+000,
       2.26160886e+000, 3.49735724e+000, 5.40832145e+000, 8.36344097e+000,
       1.29332447e+001, 2.00000000e+001, 1.00000000e+020])

In [7]:
model_dr.b0.shape

(30,)

In [8]:
model_dr.T0.shape

(31, 30, 30)

In [9]:
model_dr.Tr.shape

(31, 30, 30)

# Inspection model

The inspection model is below constructed by calling the method inspection_model(). 
Note that a specific probability of detection curve is here implemented.

The output provides: 
* Inspection model: model_dr.O

In [10]:
model_dr.inspection_model()

array([[9.99993750e-01, 6.24998047e-06],
       [9.99984085e-01, 1.59148880e-05],
       [9.99975389e-01, 2.46107132e-05],
       [9.99961942e-01, 3.80578087e-05],
       [9.99941148e-01, 5.88520727e-05],
       [9.99908992e-01, 9.10075200e-05],
       [9.99859269e-01, 1.40730743e-04],
       [9.99782382e-01, 2.17617984e-04],
       [9.99663495e-01, 3.36504953e-04],
       [9.99479676e-01, 5.20324206e-04],
       [9.99195484e-01, 8.04516183e-04],
       [9.98756168e-01, 1.24383223e-03],
       [9.98077189e-01, 1.92281133e-03],
       [9.97028122e-01, 2.97187751e-03],
       [9.95408015e-01, 4.59198532e-03],
       [9.92907858e-01, 7.09214215e-03],
       [9.89053968e-01, 1.09460324e-02],
       [9.83123734e-01, 1.68762664e-02],
       [9.74023136e-01, 2.59768635e-02],
       [9.60115535e-01, 3.98844645e-02],
       [9.38998750e-01, 6.10012497e-02],
       [9.07254464e-01, 9.27455356e-02],
       [8.60264898e-01, 1.39735102e-01],
       [7.92346392e-01, 2.07653608e-01],
       [6.977214

# Storing transitions

The transition and inspection models can be stored at the user's discretion and it is case dependent. 
The environment dynamics needs three main components:

* Initial probabilities: belief0  
* Transition probabilities: P
* Inspection model (or/and sensor model): O

In [11]:
# Initial probabilities
belief0 = np.zeros((1, 10, 30, 1)) # ((empty, n_comp, damage, empty))
belief0[0, :, :, 0] = model_dr.b0

In [12]:
# Transition model
P = np.zeros((3, 10, 31, 30, 30)) # ((action, n_comp, d_rate, damage, damage))
P[0, :, :, :, :] = model_dr.T0 # Do-nothing action
P[1, :, :, :, :] = model_dr.T0 # Inspection action
P[2, :, :, :, :] = model_dr.Tr # Repair action

In [13]:
# Inspection model
O = np.zeros((3, 10, 30, 2)) # ((action, n_comp, damage, inspect_outcome))
O[0, :, :, 0] = np.ones(30) # Do-nothing action
O[1, :, :, :] = model_dr.O # Inspection action
O[2, :, :, 0] = np.ones(30) # Repair action

In [14]:
# Store it (for instance as a numpy file)
np.savez('model.npz', belief0=belief0, P=P, O=O)